In [123]:
import pandas as pd
# baic transformer Decoder model
import torch
import torch.nn as nn
import torch.nn.functional as Fun
import torch.optim as optim
import numpy as np
import xformers.ops as xops
import math
from typing import Optional, Union
from torch import Tensor

main_df = pd.read_csv('adult.csv')
main_df.head()
DEVICE = 'cuda'

In [124]:
def POOL_preprocess(df):
    '''
    input the original dataframe, output the dataframe after preprocessing,
    change the numerical columns to categorical columns by qcut and cut
    then apply label encoding to all columns
    
    '''
    df = df.copy()
    CAT = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
    NUM = ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']
    # qcut on numerical columns
    for column in NUM:
        if column in ['educational-num','capital-gain','capital-loss','hours-per-week']:
            df[column] = pd.cut(df[column], 100)
        else:
            df[column] = pd.cut(df[column], 100)
    # make income column binary
    df['income'] = df['income'].apply(lambda x: 1 if x == '>50K' else 0)

    # lable encoding categorical columns
    from sklearn.preprocessing import LabelEncoder
    lb = LabelEncoder()
    df = df.apply(lambda x: lb.fit_transform(x))

    # make all catagory in every column unique

    # 迴圈處理多個欄位
    offset = 0
    for column in df.columns:
        df[column] = df[column].apply(lambda x: x + offset)
        offset += df[column].nunique()
    
    return df
tmp = POOL_preprocess(main_df)
tmp.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,8,78,97,164,185,199,209,220,225,229,230,253,341,437,440
1,21,78,88,174,187,197,207,217,227,229,230,253,351,437,440
2,11,76,104,170,190,197,213,217,227,229,230,253,341,437,441
3,27,78,93,178,188,197,209,217,225,229,237,253,341,437,441
4,1,74,89,178,188,199,202,220,227,228,230,253,331,437,440


In [125]:
train_size = 4*48842//5
test_size = 48842//5
train_pool = main_df[test_size:]
test_pool = main_df[:test_size]
print('total data num:' , main_df.shape[0])
print('trian data num:' , train_pool.shape[0])
print('test data num:' , test_pool.shape[0])

total data num: 48842
trian data num: 39074
test data num: 9768


In [126]:
# notations
#   node: number of all nodes = L + S + C + F
#   L: number of lable nodes
#   S: number of sample nodes
#   C: number of catagory nodes
#   F: number of field(column) nodes
#   hidden: number of hidden representation

# data size = (node, hidden)
# mask size = (node, node - L) without lable nodes
#             for each node, real mask = cat[mask,(node,L)] = (node, node)
#             cannot see it's label node

# use nn.transformerDecoder(data,mask) to get the output
# use the above output as input of MLP to predict the lable   

In [127]:
class HGNN_DataSet():
    def __init__(self,
                 data_df : pd.DataFrame,
                 split_ratio : float ,
                 label_column : str,
                 ):
        test_size = math.ceil(data_df.shape[0] * (1-split_ratio))
        train_pool = data_df[test_size:]
        test_pool = data_df[:test_size]
        print('total data num:' , data_df.shape[0])
        print('trian data num:' , train_pool.shape[0])
        print('test data num:' , test_pool.shape[0])
        
        # to-dos:
        # train
        #   
        
        TARGET_POOL = POOL_preprocess(train_pool)
        LABEL_COLUMN = label_column

        # cut feature and lable
        FEATURE_POOL = TARGET_POOL.drop(LABEL_COLUMN, axis=1)
        LABEL_POOL = TARGET_POOL[LABEL_COLUMN]

        # trasform label into one-hot
        from sklearn.preprocessing import OneHotEncoder
        enc = OneHotEncoder()
        LABEL_POOL = enc.fit_transform(LABEL_POOL.values.reshape(-1,1)).toarray()

        # L: number of lable nodes
        # the last node of Lable nodes is served as unknown lable node
        L = LABEL_POOL.shape[1] + 1

        # S: number of sample nodes
        S = FEATURE_POOL.shape[0] + 1
        # the last node of sample nodes is served as infering node

        # C: number of catagory nodes
        C = FEATURE_POOL.apply(lambda x: x.nunique()).sum() # total_unique_labels
        C_POOL = sorted(list(set(FEATURE_POOL.values.flatten())))
        # the last node of catagory nodes is served as Unseen node
        C += 1
        C_POOL.append(C_POOL[-1] + 1)

        # F: number of field(column) nodes
        F = FEATURE_POOL.shape[1]

        nodes_num = {'L':L, 'S':S, 'C':C, 'F':F}
        print('node_nums', nodes_num)
        print('total', L+S+C+F, 'nodes')
        
        self.TARGET_POOL = TARGET_POOL
        self.LABEL_COLUMN = LABEL_COLUMN
        self.FEATURE_POOL = FEATURE_POOL
        self.LABEL_POOL = LABEL_POOL
        self.C_POOL = C_POOL   
        self.nodes_num = nodes_num

        
        self.make_input_tensor()
        # self.get_sample(10)        
        self.make_mask_all()
        
        # self.make_mask()
        
        
    def make_mask(self,
                  sample_indices: Optional[list] = None,
                ):
        L, S, C, F = self.nodes_num['L'], self.nodes_num['S'], self.nodes_num['C'], self.nodes_num['F']
        sample_indices = [int(x) for x in sample_indices]
        sample_size = len(sample_indices)
        masked_POOL = self.TARGET_POOL.iloc[sample_indices]
        # caculate masking
        masks = {}
        
    
        
        # label to sample 
        tmp = torch.zeros([math.ceil(sample_size/8) * 8, math.ceil(L/8) * 8], dtype=torch.float).to(DEVICE)
        # tmp = torch.zeros([math.ceil(sample_size/8) * 8, math.ceil(L/8) * 8], dtype=torch.float).to(DEVICE)[:len(sample_indices),:L]
        label_ids = self.TARGET_POOL[self.LABEL_COLUMN].unique()
        for i, value_df in enumerate(masked_POOL[self.LABEL_COLUMN]):
            for j, value_label in enumerate(label_ids):
                if value_label == value_df:
                    tmp[i][j] = 1
                    break
        # tmp_ = torch.zeros_like(self.MASKS_FULL['L2S'].detach().clone()[sample_indices])
        # tmp.copy_(self.MASKS_FULL['L2S'].detach().clone()[sample_indices][:,:L])
        # tmp = Tensor.contiguous(self.MASKS_FULL['L2S'].detach().clone()[sample_indices])
        masks['L2S'] = Tensor.contiguous(tmp)

        # sample to catagory
        tmp = torch.zeros([math.ceil(C/8) * 8, math.ceil(sample_size/8) * 8], dtype=torch.float).to(DEVICE)
        # tmp = torch.zeros([math.ceil(C/8) * 8, math.ceil(sample_size/8) * 8], dtype=torch.float).to(DEVICE)[:C,:len(sample_indices)]
        
        tmp_df = masked_POOL.drop(self.LABEL_COLUMN, axis=1)
        for i, value_df in enumerate(tmp_df.values):
            for j, value in enumerate(value_df):
                tmp[value][i] = 1
        
        # tmp_ = torch.zeros_like(self.MASKS_FULL['S2C'].detach().clone()[:,sample_indices])
        # tmp.copy_(self.MASKS_FULL['S2C'].detach().clone()[:,sample_indices][:C,:])
        # print("err",tmp.is_contiguous())
        # tmp = Tensor.contiguous(self.MASKS_FULL['S2C'].detach().clone()[:,sample_indices])
        masks['S2C'] = Tensor.contiguous(tmp)

        # catagory to field
        masks['C2F'] = Tensor.contiguous(self.MASKS_FULL['C2F'])
        
        self.MASKS = masks
        self.nodes_num['K'] = sample_size
        
    def make_mask_all(self,
                  sample_indices: Optional[torch.tensor] = None,
                ):
        L, S, C, F = self.nodes_num['L'], self.nodes_num['S'], self.nodes_num['C'], self.nodes_num['F']
        # caculate masking
        masks = {}
        
        # label to sample 
        tmp = torch.zeros([math.ceil(S/8) * 8, math.ceil(L/8) * 8], dtype=torch.float)
        label_ids = self.TARGET_POOL[self.LABEL_COLUMN].unique()
        for i, value_df in enumerate(self.TARGET_POOL[self.LABEL_COLUMN]):
            for j, value_label in enumerate(label_ids):
                if value_label == value_df:
                    tmp[i][j] = 1
                    break
        masks['L2S'] = tmp

        # sample to catagory
        tmp = torch.zeros([math.ceil(C/8) * 8, math.ceil(S/8) * 8], dtype=torch.float)
        tmp_df = self.TARGET_POOL.drop(self.LABEL_COLUMN, axis=1)
        for i, value_df in enumerate(tmp_df.values):
            for j, value in enumerate(value_df):
                tmp[value][i] = 1
        masks['S2C'] = tmp

        # catagory to field
        tmp = torch.zeros([math.ceil(F/8) * 8, math.ceil(C/8) * 8], dtype=torch.float)
        unique_items = [(self.TARGET_POOL[column].unique()) for column in (self.TARGET_POOL.columns)]
        for i in range(F):
            for j in (unique_items[i]):
                tmp[i][j] = 1
        masks['C2F'] = tmp
        
        
        self.MASKS = masks
        self.MASKS_FULL = masks
    
    def make_input_tensor(self):
        # make input tensor
        L, S, C, F = self.nodes_num['L'], self.nodes_num['S'], self.nodes_num['C'], self.nodes_num['F']
        # L
        L_input = torch.eye(L).to(DEVICE)
        print('L_input', L_input.type(), L_input.shape)
        # S
        S_input = torch.tensor(np.vstack([self.FEATURE_POOL.values, np.zeros(F)])).to(DEVICE)
        print('S_input', S_input.type(), S_input.shape)
        # C random init
        C_input = torch.tensor(np.diag(self.C_POOL)).to(DEVICE)
        print('C_input', C_input.type(), C_input.shape)
        # F random init
        F_input = torch.eye(F).to(DEVICE)
        print('F_input', F_input.type(), F_input.shape)
        print(L_input.type())
        # 
        self.INPUTS = (L_input, S_input, C_input, F_input)
        self.INPUT_DIMS = (L_input.size(1), S_input.size(1), C_input.size(1), F_input.size(1))
        
    def get_sample(self, sample_size, inculde = []):
        # get K samples from S
        # return sample node mask
        S = self.nodes_num['S']
        
        # inculde specific nodes (e.g. query nodes), while remaining sample_size
        # -1 is infering node
        sample_indices = sorted(torch.randperm(S-1)[:sample_size - len(inculde)])
        if inculde is not []:
            while inculde in sample_indices:
                sample_indices = (torch.randperm(S-1)[:sample_size - len(inculde)])
            # add inculde nodes into sample_indices
            for node in inculde:
                sample_indices.append(node)
            sample_indices = sorted(sample_indices)
        # update mask
        self.make_mask(sample_indices)
        
        # update input tensor
        L_input, S_input, C_input, F_input = self.INPUTS
        # S_input_masked = S_input[int(sample_indices)]
        S_input_masked = torch.index_select(S_input, 0, torch.tensor([int(x) for x in sample_indices]).to(DEVICE))
        self.MASKED_INPUTS = (L_input, S_input_masked, C_input, F_input)   
        return sample_indices
            
Train_data = HGNN_DataSet( main_df, 0.8, 'income')


total data num: 48842
trian data num: 39073
test data num: 9769


node_nums {'L': 3, 'S': 39074, 'C': 437, 'F': 14}
total 39528 nodes
L_input torch.cuda.FloatTensor torch.Size([3, 3])
S_input torch.cuda.DoubleTensor torch.Size([39074, 14])
C_input torch.cuda.LongTensor torch.Size([437, 437])
F_input torch.cuda.FloatTensor torch.Size([14, 14])
torch.cuda.FloatTensor


In [128]:
sum(Train_data.MASKS['S2C'])

tensor([14., 14., 14.,  ...,  0.,  0.,  0.])

In [129]:
Train_data.get_sample(10)


[tensor(9831),
 tensor(10307),
 tensor(11251),
 tensor(14011),
 tensor(14157),
 tensor(15303),
 tensor(16530),
 tensor(29717),
 tensor(30752),
 tensor(38748)]

In [130]:
from torch import Tensor
from typing import Optional, Any, Union, Callable

class CustomTransformerDecoderLayer(nn.TransformerDecoderLayer):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1, activation='relu'):
        super().__init__(d_model, nhead, dim_feedforward, dropout, activation)
        # remove defined modules
        delattr(self, 'self_attn')
        delattr(self, 'norm1')
        delattr(self, 'dropout1')
    
    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        x = tgt
        if self.norm_first:
            # x = x + self._sa_block(self.norm1(x), tgt_mask, tgt_key_padding_mask)
            x = x + self._mha_block(self.norm2(x), memory, memory_mask)
            x = x + self._ff_block(self.norm3(x))
        else:
            # x = self.norm1(x + self._sa_block(x, tgt_mask, tgt_key_padding_mask))
            x = self.norm2(x + self._mha_block(x, memory, memory_mask))
            # x =  x + self._mha_block(x, memory, memory_mask, memory_key_padding_mask)
            # x = self.norm3(x + self._ff_block(x))

        return x
    def _mha_block(self, x: Tensor, mem: Tensor,
                   attn_mask: Optional[Tensor],) -> Tensor:
        # random_mask = torch.cuda.FloatTensor(mem.shape[0], 1).uniform_() > 0.8
        # print(sum(sum(attn_mask)))
        x = xops.memory_efficient_attention(x, mem, mem, attn_mask)
        return self.dropout2(x)


In [131]:
tmp = Train_data.MASKS['L2S'].clone()

In [132]:
# baic transformer decoder model
import torch
import torch.nn as nn
import torch.nn.functional as Fun
from tqdm import trange

class TransformerDecoderModel(nn.Module):
    def __init__(self, 
                 target_dataset, 
                 num_layers, 
                 embedding_dim,  
                 subgraph_masked: Optional[bool] = False,
                 K : Optional[int] = 10,
                 ):
        super(TransformerDecoderModel, self).__init__()

        L_dim, S_dim, C_dim, F_dim = target_dataset.INPUT_DIMS
        
        # 目前b卡在embedding的怎麼用
        # Catagory_embedding => 數值類Qcut後用linear來做embedding, 類別用nn.Embedding

        self.Lable_embedding = nn.Linear(L_dim, embedding_dim, dtype=torch.float)
        self.Sample_embedding = nn.Linear(S_dim, embedding_dim, dtype=torch.float)
        self.Catagory_embedding = nn.Linear(C_dim, embedding_dim, dtype=torch.float)
        self.Field_embedding = nn.Linear(F_dim, embedding_dim, dtype=torch.float)
        
        self.transformer_decoder = nn.TransformerDecoder(
            CustomTransformerDecoderLayer(embedding_dim,  nhead = 1 ),
            num_layers
        )
        
        # downstream task
        self.MLP = nn.Sequential(
            nn.Linear(embedding_dim, 2),
            nn.Softmax(dim=2),
            # nn.Sigmoid(),
        )
        
        self.subgraph_masked = subgraph_masked
        if subgraph_masked: 
            self.K = K
        else:
            # init mask
            target_dataset.make_mask_all()
        
        self.tmpmask_L2S = target_dataset.MASKS['L2S'].clone().to(DEVICE)


    def forward(self, 
                target_dataset: HGNN_DataSet, 
                ):
        L, S, C, F = target_dataset.nodes_num['L'], target_dataset.nodes_num['S'], target_dataset.nodes_num['C'], target_dataset.nodes_num['F']
        
        if self.subgraph_masked: 
            target_dataset.get_sample(self.K)
            L_input, S_input, C_input, F_input = target_dataset.MASKED_INPUTS
            masks = target_dataset.MASKS
            K = target_dataset.nodes_num['K']
            S_ = K
        else:
            target_dataset.make_input_tensor()
            L_input, S_input, C_input, F_input = target_dataset.INPUTS
            target_dataset.make_mask_all()
            masks = target_dataset.MASKS
            S_ = S
            
    
        L_embedded = self.Lable_embedding(L_input.float()).unsqueeze(0)
        S_embedded = self.Sample_embedding(S_input.float()).unsqueeze(0)
        C_embedded = self.Catagory_embedding(C_input.float()).unsqueeze(0)
        F_embedded = self.Field_embedding(F_input.float()).unsqueeze(0)
        
        for mask in masks.keys():
            masks[mask] = masks[mask].to(DEVICE)
        
        # propagate steps: L→S→C→F
        #                  L←S←C←
        # more steps more menory usage
        PROPAGATE_STEPS = 1
        for i in range(PROPAGATE_STEPS):
            S_embedded = self.transformer_decoder(S_embedded,L_embedded, 
                                                memory_mask = self.tmpmask_L2S[:S_,:L]) 
            C_embedded = self.transformer_decoder(C_embedded,S_embedded,
                                                memory_mask = masks['S2C'][:C,:S_])
            F_embedded = self.transformer_decoder(F_embedded,C_embedded,
                                                memory_mask = masks['C2F'][:F,:C])
            C_embedded = self.transformer_decoder(C_embedded,F_embedded,
                                                memory_mask = Tensor.contiguous(masks['C2F'].transpose(0, 1))[:C,:F])
            S_embedded = self.transformer_decoder(S_embedded,C_embedded,
                                                memory_mask = Tensor.contiguous(masks['S2C'].transpose(0, 1))[:S_,:C])
            L_embedded = self.transformer_decoder(L_embedded,S_embedded, 
                                                memory_mask = Tensor.contiguous(self.tmpmask_L2S.transpose(0, 1))[:L,:S_])
        
        
        output = self.MLP(S_embedded)[0]
        return output

# 測試模型
num_layers = 1  # TransformerDecoder 的層數
embedding_dim = 2*128  # 嵌入維度
hidden_dim = 64  

print('input_dims', Train_data.INPUT_DIMS)
model = TransformerDecoderModel(Train_data, num_layers, embedding_dim, subgraph_masked = True, K = 10).to(DEVICE)
outputs = model(Train_data)

print("模型輸出的大小:", outputs.shape)

input_dims (3, 14, 437, 14)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
模型輸出的大小: torch.Size([10, 2])


In [133]:
Train_data.MASKS

{'L2S': tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0'),
 'S2C': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'),


In [134]:
output_label = torch.argmax(outputs, dim=1)

In [135]:
# training
from torch import autograd
from torcheval.metrics.aggregation.auc import AUC

def train(model, datset):
    LABEL_POOL = datset.LABEL_POOL
    
    
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    epochs = 100
    for epoch in range(epochs):
        
        # train
        model.train()
        Original_L2S_mask = datset.MASKS['L2S'].clone()
        loss_log = []
        for index in trange(datset.nodes_num['S']-1): # query through all sample nodes (not infering node)
            # for all query, input = sample K + 1 query smaple
            sample_indices = datset.get_sample(10, inculde = [index])
            
            
            # modify the mask to mask out the queries node's edge to it's label node
            L = datset.nodes_num['L']
            query_index = sample_indices.index(index)
            datset.tmpmask_L2S = Original_L2S_mask.clone()
            datset.tmpmask_L2S[query_index] = 0
            datset.tmpmask_L2S[query_index][L-1] = 1 # make it as unseen label
            
            
            optimizer.zero_grad()
            outputs = model(datset)
            # for trainning, only there is no infering node

            # caculate loss
            if model.subgraph_masked:
                LABEL_POOL_ = LABEL_POOL[sample_indices]
            else:
                LABEL_POOL_ = LABEL_POOL
            batch_loss = criterion(outputs, torch.tensor(LABEL_POOL_).to(DEVICE))
            loss_log.append(batch_loss.item())
            # backpropagation
            batch_loss.backward()
            optimizer.step()
        
        # eval on last sample node
        output_label = torch.argmax(outputs, dim=1)
        
        TRUE = torch.argmax(torch.tensor(LABEL_POOL_).to(DEVICE), dim=1)
        print(len(output_label), len(TRUE))
        metric = AUC()
        y_pred_first = torch.tensor([x[0] for x in outputs]).to(DEVICE)
        metric.update(TRUE,y_pred_first)
        epoch_AUC = float(metric.compute())

        epoch_acc = torch.sum(output_label == torch.argmax(torch.tensor(LABEL_POOL_).to(DEVICE), dim=1)).item() / len(output_label)
        
        epoch_loss = sum(loss_log)/(datset.nodes_num['S']-1)

        print(f"Epoch{epoch+1}/{epochs} | Loss: {epoch_loss} | Acc: {epoch_acc} | AUC: {epoch_AUC}")
model = TransformerDecoderModel(Train_data, num_layers, embedding_dim, subgraph_masked = True).to(DEVICE)
train(model, Train_data)

  0%|          | 14/39073 [00:00<04:48, 135.26it/s]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
 

  0%|          | 28/39073 [00:00<05:57, 109.30it/s]

  0%|          | 41/39073 [00:00<05:34, 116.54it/s]

tensor([[0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
 

...

  9%|▉         | 3698/39073 [00:29<04:48, 122.62it/s]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
 

  9%|▉         | 3707/39073 [00:29<04:37, 127.37it/s]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([437, 10])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
 

KeyboardInterrupt: 

In [ ]:
# self.Sample_embedding = nn.Linear(S_dim, embedding_dim, dtype=torch.half)
# self.Catagory_embedding = nn.Linear(C_dim, embedding_dim, dtype=torch.half)
# S_embedded = self.Sample_embedding(S_input.half()).unsqueeze(0)
# C_embedded = self.Catagory_embedding(C_input.half()).unsqueeze(0)
# xops.memory_efficient_attention(C_embedded, S_embedded, S_embedded,attn_mask)[0]